In [1]:
import ufl
import numpy as np
from mpi4py import MPI
from dolfinx.mesh import create_interval, create_rectangle, CellType
from dolfinx.fem import form, assemble_scalar, functionspace, Constant, Function
from dolfinx.fem.petsc import create_vector, create_matrix, assemble_vector, assemble_matrix
from petsc4py import PETSc
import gmsh
from dolfinx import io

np.set_printoptions(precision=3, suppress=False, formatter={'float': '{:0.1e}'.format})

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 0) 

p1 = gmsh.model.geo.addPoint(0, 0, 0, 1.0) # x, y, z, tamaño característico
p2 = gmsh.model.geo.addPoint(1, 0, 0, 1.0)
linea = gmsh.model.geo.addLine(p1, p2)


gmsh.model.geo.synchronize()

gmsh.model.add_physical_group(1, [linea], 2)

# Sincronizar para aplicar los cambios
gmsh.model.geo.synchronize()
for line in gmsh.model.getEntities(1):
    line_tag = line[1]
    gmsh.model.mesh.set_transfinite_curve(line_tag, 2)

gmsh.model.geo.synchronize()

gmsh.model.mesh.generate(dim=1)

mesh, markers, facets = io.gmshio.model_to_mesh(gmsh.model, MPI.COMM_WORLD, 0, gdim=2)

# Define cross-sectional areas
A1 = np.pi*4e-6
A2 = np.pi*16e-6

rho1 = 2.7e3
rho2 = 19.3e3

E1 = 70e9
E2 = 411e9

print(f"Area difference (A2 - A1): {A2 - A1}")

# Create function spaces
V = functionspace(mesh, ("CG", 1, (2,)))  # Continuous Galerkin space for solution
V0 = functionspace(mesh, ("DG", 0))  # Discontinuous Galerkin space for parameter

# Create control variable function
x_A = Function(V0)
x_A.x.array[:] = 1  # Initialize with a value between 0 and 1
A = (A2 - A1) * x_A + A1

# Create control variable function
x_rho= Function(V0)
x_rho.x.array[:] = 1  # Initialize with a value between 0 and 1
rho = (rho2 - rho1) * x_rho + rho1

# Create control variable function
x_E= Function(V0)
x_E.x.array[:] = 1  # Initialize with a value between 0 and 1
E = (E2 - E1) * x_E + E1

# Define trial and test functions
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Define bilinear form (mass matrix)
a = ufl.inner(A*rho*u, v)*ufl.dx

epsilon = lambda u: ufl.sym(ufl.grad(u))
sigma = lambda u: ufl.tr(epsilon(u)) * ufl.Identity(len(u))

k = E * A *ufl.inner(sigma(u), epsilon(v)) * ufl.dx

# Assemble mass matrix
M = create_matrix(form(a))
assemble_matrix(M, form(a))
M.assemble()

print("Mass matrix:")
print(np.array(M[:,:]))

# Calculate derivative with respect to control variable
se = Function(V0)
se.x.array[:] = 0
se.x.array[0] = 1
da = se * ufl.inner(A*rho*u, v)*ufl.dx
da_form = form(ufl.adjoint(ufl.diff(da, x_A)))
dM = create_matrix(da_form)
assemble_matrix(dM, da_form)
dM.assemble()

print("\nDerivative of mass matrix with respect to control variable:")
print(np.array(dM[:,:]))

Area difference (A2 - A1): 3.7699111843077517e-05
Mass matrix:
[[0.323+0.j 0.   +0.j 0.162+0.j 0.   +0.j]
 [0.   +0.j 0.323+0.j 0.   +0.j 0.162+0.j]
 [0.162+0.j 0.   +0.j 0.323+0.j 0.   +0.j]
 [0.   +0.j 0.162+0.j 0.   +0.j 0.323+0.j]]

Derivative of mass matrix with respect to control variable:
[[0.243+0.j 0.   +0.j 0.121+0.j 0.   +0.j]
 [0.   +0.j 0.243+0.j 0.   +0.j 0.121+0.j]
 [0.121+0.j 0.   +0.j 0.243+0.j 0.   +0.j]
 [0.   +0.j 0.121+0.j 0.   +0.j 0.243+0.j]]


In [2]:
dk_form = form(ufl.diff(a, x_A))
dK = create_matrix(dk_form)
assemble_matrix(dK, dk_form)
dK.assemble()

print(np.array(dK[:,:]))

[[0.243+0.j 0.   +0.j 0.121+0.j 0.   +0.j]
 [0.   +0.j 0.243+0.j 0.   +0.j 0.121+0.j]
 [0.121+0.j 0.   +0.j 0.243+0.j 0.   +0.j]
 [0.   +0.j 0.121+0.j 0.   +0.j 0.243+0.j]]
